In [2]:
import random
import copy
import time

In [3]:
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        The move is valid if:
        - The selected pit number is in range (1-indexed by player perspective)
        - The selected pit contains at least one stone
        """
        pit -= 1
        if self.current_player == 1:
            pit_index = self.p1_pits_index[0] + pit
            if pit_index < self.p1_pits_index[0] or pit_index > self.p1_pits_index[1]:
                return False
        else:
            pit_index = self.p2_pits_index[0] + pit
            if pit_index < self.p2_pits_index[0] or pit_index > self.p2_pits_index[1]:
                return False
        return self.board[pit_index] > 0

        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        if self.current_player == 1:
            valid_pits = [i - self.p1_pits_index[0] + 1 
                        for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1) 
                        if self.board[i] > 0]
        else:
            valid_pits = [i - self.p2_pits_index[0] + 1
                        for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1)
                        if self.board[i] > 0]

        if not valid_pits:
            return None

        return random.choice(valid_pits)
    
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit.
        It performs the following:
        1. Validates the chosen pit (1-indexed from player's perspective).
        2. Checks if the game has ended.
        3. Distributes stones counter-clockwise, skipping opponent's mancala.
        4. Applies capture rule if last stone lands in an empty pit on player's side.
        5. Records the move.
        6. Switches to the other player's turn (no extra turns).
        """
        if not self.valid_move(pit):
            print("INVALID MOVE")
            return self.board
        if self.winning_eval():
            print("GAME OVER")
            return self.board
            
        pit -= 1
        if self.current_player == 1:
            pit_index = self.p1_pits_index[0] + pit
        else:
            pit_index = self.p2_pits_index[0] + pit

        stones = self.board[pit_index]
        self.board[pit_index] = 0
        index = pit_index
        while stones > 0:
            index = (index + 1) % len(self.board)
            if self.current_player == 1 and index == self.p2_mancala_index:
                continue
            if self.current_player == 2 and index == self.p1_mancala_index:
                continue
            self.board[index] += 1
            stones -= 1

        if self.current_player == 1 and self.p1_pits_index[0] <= index <= self.p1_pits_index[1] and self.board[index] == 1:
            opposite = self.p2_pits_index[1] - (index - self.p1_pits_index[0])
            self.board[self.p1_mancala_index] += self.board[opposite] + 1
            self.board[opposite] = 0
            self.board[index] = 0
        elif self.current_player == 2 and self.p2_pits_index[0] <= index <= self.p2_pits_index[1] and self.board[index] == 1:
            opposite = self.p1_pits_index[1] - (index - self.p2_pits_index[0])
            self.board[self.p2_mancala_index] += self.board[opposite] + 1
            self.board[opposite] = 0
            self.board[index] = 0

        self.moves.append((self.current_player, pit + 1))
        self.current_player = 2 if self.current_player == 1 else 1
        return self.board

    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        p1_empty = all(self.board[i] == 0 for i in range(self.p1_pits_index[0], self.p1_pits_index[1]+1))
        p2_empty = all(self.board[i] == 0 for i in range(self.p2_pits_index[0], self.p2_pits_index[1]+1))
        if not (p1_empty or p2_empty):
            return False
        if not p1_empty:
            for i in range(self.p1_pits_index[0], self.p1_pits_index[1]+1):
                self.board[self.p1_mancala_index] += self.board[i]
                self.board[i] = 0
        if not p2_empty:
            for i in range(self.p2_pits_index[0], self.p2_pits_index[1]+1):
                self.board[self.p2_mancala_index] += self.board[i]
                self.board[i] = 0
        return True

    def utility(self, max_player=1):
        """
        Calculates the utility of the board for the given max_player.
        Utility = stones in Max's Mancala - stones in Min's Mancala
        """
        if max_player == 1: #check if AI is player 1
            return self.board[self.p1_mancala_index] - self.board[self.p2_mancala_index]
        else:
            return self.board[self.p2_mancala_index] - self.board[self.p1_mancala_index]


In [4]:
def simulateGames(pits_p_player, stones_p_pit, n=100):
    i = 0
    p1_games_won = 0
    p2_games_won = 0
    tied_games = 0
    p1_turns = 0
    p2_turns = 0
    while(i < n):
        game = Mancala(pits_p_player, stones_p_pit)
        while(game.winning_eval() == False):
            move = game.random_move_generator()
            if move is not None:
                if game.current_player == 1:
                    p1_turns += 1
                else:
                    p2_turns += 1
                game.play(move)
        
        p1_score = game.board[game.p1_mancala_index]
        p2_score = game.board[game.p2_mancala_index]
        if p1_score > p2_score:
            p1_games_won +=1
        elif p2_score > p1_score:
            p2_games_won +=1
        else:
            tied_games +=1    
        i+=1
    
    print("Number of games won by player 1: ", p1_games_won)
    print("Number of games won by player 2: ", p2_games_won)
    print("Number of games tied: ", tied_games)
    print("Average number of moves by Player 1: ", p1_turns/n)
    print("Average number of moves by Player 2: ", p2_turns/n)
    print("Percentage of first move winning: ", (p1_games_won / (n- tied_games))*100, '%')

In [5]:
simulateGames(6,4,100)

Number of games won by player 1:  56
Number of games won by player 2:  37
Number of games tied:  7
Average number of moves by Player 1:  21.61
Average number of moves by Player 2:  21.13
Percentage of first move winning:  60.215053763440864 %


In [6]:
class AIPlayer:
    def __init__(self, player_num, strategy="minimax", depth=4):
        self.player_num = player_num
        self.strategy = strategy
        self.depth = depth

    def choose_move(self, game: Mancala):
        if self.strategy == "minimax":
            _, move = self.minimax(game, self.depth, game.current_player == self.player_num)
            return move
        elif self.strategy == "alpha beta":
            _, move = self.alphaBeta(game, self.depth, float('-inf'), float('inf'), game.current_player == self.player_num)
            return move
        else:
            return game.random_move_generator()

    def get_valid_moves(self, game):
        if game.current_player == 1:
            return [i - game.p1_pits_index[0] + 1
                    for i in range(game.p1_pits_index[0], game.p1_pits_index[1] + 1)
                    if game.board[i] > 0]
        else:
            return [i - game.p2_pits_index[0] + 1
                    for i in range(game.p2_pits_index[0], game.p2_pits_index[1] + 1)
                    if game.board[i] > 0]

    def minimax(self, game, depth, maximizer):
        if depth == 0 or game.winning_eval():
            return game.utility(self.player_num), None
        best_move = None
        if maximizer == True:
            max_eval = float('-inf')
            for move in self.get_valid_moves(game):
                new_game = copy.deepcopy(game)
                new_game.play(move)
                eval, _ = self.minimax(new_game, depth-1, False)
                if eval > max_eval:
                    max_eval = eval
                    best_move = move
            return max_eval, best_move
        else:
            min_eval = float('inf')
            for move in self.get_valid_moves(game):
                new_game = copy.deepcopy(game)
                new_game.play(move)
                eval, _ = self.minimax(new_game, depth-1, True)
                if eval < min_eval:
                    min_eval = eval
                    worst_move = move
            return min_eval, worst_move

    
    def alphaBeta(self, game, depth, alpha, beta, maximizer): #alpha-beta implementation
    #base case, return value found by function when we reach bottom/leaf nodes
        if depth == 0 or game.winning_eval(): #if we reach bottom, or game is over, return the utility found
            return game.utility(self.player_num), None

        best_move = None #placeholder for best move to be found later

        # max function for AI Player 
        if maximizer:
            max_eval = float('-inf')  #set as -inf because, thats the worst possible value

            #iterate through possible moves
            for move in self.get_valid_moves(game):
                #create a copy of the current game and try current move
                new_game = copy.deepcopy(game)
                new_game.play(move)

                #recursively evaluate resulting position in term of opponents position
                eval_val, _ = self.alphaBeta(new_game, depth-1, alpha, beta, False)

                #if found better move, update best_move
                if eval_val > max_eval:
                    max_eval = eval_val
                    best_move = move

                #update alpha so that maximizer works best
                alpha = max(alpha, eval_val)

                # Pruning: if beta <= alpha, opponent won't allow this position, so stop exploring. (AB pruning rules)
                if beta <= alpha:
                    break #tree "pruned"

            return max_eval, best_move

        #min function for opponent
        else:
            min_eval = float('inf')  #set as inf so that it can be updated initially

            #iterate through each possible move
            for move in self.get_valid_moves(game):
                #create copy of game and try current move
                new_game = copy.deepcopy(game)
                new_game.play(move)

                #recursively evaluate resulting position for AI player
                eval_val, _ = self.alphaBeta(new_game, depth-1, alpha, beta, True)

                #update best move if better move is found
                if eval_val < min_eval:
                    min_eval = eval_val
                    best_move = move

                #update beta as minimizer function
                beta = min(beta, eval_val)

                # Pruning: if beta (opponent's best) <= alpha (AB pruning rules)
                if beta <= alpha:
                    break  #"pruning"

            return min_eval, best_move

In [16]:
def simulateAIvsRandom(pits_p_player, stones_p_pit, ai_player=1, depth=3, n=100, strategy_in="minimax"):

    #validate strategy inputs
    if strategy_in != "alpha beta" and strategy_in != "minimax":
        print(f"Please enter a valid strategy ('minimax' or 'alpha beta')")
        return -1
    
    #counters for end of game stats
    ai_games_won = 0
    random_games_won = 0
    tied_games = 0
    ai_turns = 0
    random_turns = 0
    start_time = time.time()
    
    #create ai player that uses inputted strat
    ai = AIPlayer(ai_player, strategy=strategy_in, depth=depth)
    
    for i in range(n):
        #init game board (reality board)
        game = Mancala(pits_p_player, stones_p_pit)
        
        #while loop to play until game is over
        while not game.winning_eval():
            #when its the AI turn
            if game.current_player == ai_player: #if ais turn, choose best move and execute it if found
                move = ai.choose_move(game)
                if move is not None:
                    ai_turns += 1
                    game.play(move)
            #random players turn
            else:
                move = game.random_move_generator() #choose random move and execute it
                if move is not None:
                    random_turns += 1
                    game.play(move)
        
        #choose winner
        p1_score = game.board[game.p1_mancala_index]
        p2_score = game.board[game.p2_mancala_index]
        
        if p1_score > p2_score and ai_player == 1: #if ai won, increment ai counter (for each player)
            ai_games_won += 1
        elif p2_score > p1_score and ai_player == 2:
            ai_games_won += 1
        elif p1_score > p2_score and ai_player == 2: #if random player won, increment random counter (for each player)
            random_games_won += 1
        elif p2_score > p1_score and ai_player == 1:
            random_games_won += 1
        else: #if neither won, increment tied counter
            tied_games += 1
            
        #sanity check so i know its running
        if (i+1) % 10 == 0:
            print(f"Completed {i+1} games...")
    
    #display results
    ai_player_name = f"Player {ai_player} (Alpha-Beta AI)"
    random_player_name = f"Player {2 if ai_player == 1 else 1} (Random)"
    end_time = time.time()
    total_time = end_time - start_time
    
    print("\n===== ", strategy_in, " vs Random Game Simulation Results =====")
    print(f"Total games: {n}")
    print(f"{ai_player_name} wins: {ai_games_won} ({ai_games_won/n*100:.1f}%)")
    print(f"{random_player_name} wins: {random_games_won} ({random_games_won/n*100:.1f}%)")
    print(f"Tied games: {tied_games} ({tied_games/n*100:.1f}%)")
    print(f"Average moves by {ai_player_name}: {ai_turns/n:.1f}")
    print(f"Average moves by {random_player_name}: {random_turns/n:.1f}")
    print(f"Total simulation time: {total_time:.2f} seconds")
    
    if ai_games_won + random_games_won > 0: #if not all ties, print amount
        print(f"AI win rate: {(ai_games_won / (ai_games_won + random_games_won))*100:.1f}%")
    
    return 1

In [17]:
simulateAIvsRandom(6, 4, ai_player=2, depth=5, n=100, strategy_in="minimax")

Completed 10 games...
Completed 20 games...
Completed 30 games...
Completed 40 games...
Completed 50 games...
Completed 60 games...
Completed 70 games...
Completed 80 games...
Completed 90 games...
Completed 100 games...

=====  minimax  vs Random Game Simulation Results =====
Total games: 100
Player 2 (Alpha-Beta AI) wins: 95 (95.0%)
Player 1 (Random) wins: 5 (5.0%)
Tied games: 0 (0.0%)
Average moves by Player 2 (Alpha-Beta AI): 14.3
Average moves by Player 1 (Random): 15.0
Total simulation time: 141.10 seconds
AI win rate: 95.0%


1

In [18]:
simulateAIvsRandom(6, 4, ai_player=2, depth=5, n=100, strategy_in="alpha beta")

Completed 10 games...
Completed 20 games...
Completed 30 games...
Completed 40 games...
Completed 50 games...
Completed 60 games...
Completed 70 games...
Completed 80 games...
Completed 90 games...
Completed 100 games...

=====  alpha beta  vs Random Game Simulation Results =====
Total games: 100
Player 2 (Alpha-Beta AI) wins: 97 (97.0%)
Player 1 (Random) wins: 2 (2.0%)
Tied games: 1 (1.0%)
Average moves by Player 2 (Alpha-Beta AI): 13.9
Average moves by Player 1 (Random): 14.5
Total simulation time: 33.93 seconds
AI win rate: 98.0%


1